In [1]:
from random import choice

In [2]:
from ga4stpg.graph.reader import read_problem
from ga4stpg.graph import UGraph
from ga4stpg.tree.generate import GenerateBasedPrimRST

from ga4stpg.edgeset import UEdge

In [3]:
instance_file = 'steinb5.txt'

stpg = read_problem('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary', instance_file)

In [4]:
generate = GenerateBasedPrimRST(stpg)

In [5]:
red = generate()
blue = generate()

In [6]:
red.edges == blue.edges

False

In [7]:
g_union  = UGraph()
g_common = UGraph()
g_star   = UGraph()

In [8]:
for v, u in red.gen_undirect_edges():
    g_union.add_edge(v,u)
    if blue.has_edge(v,u):
        g_common.add_edge(v,u)
    else:
        g_star.add_edge(v, u)

for v,u in blue.gen_undirect_edges():
    g_union.add_edge(v,u)
    if not red.has_edge(v, u):
        g_star.add_edge(v, u)

In [9]:
common_nodes = set()

for v in g_star.vertices:
    if red.has_node(v) and blue.has_node(v):
        common_nodes.add(v)

In [10]:
print(len(common_nodes))
print(len(g_star.vertices))

len(common_nodes) == len(g_star.vertices)

38
38


True

In [11]:
g_star_nodes = set(g_star.vertices)

g_star_nodes == common_nodes

True

### Utilizando DFS para classificar as arestas em comuns, vermelhas ou azuis


In [12]:
vertices = set(g_union.vertices)
print(len(vertices))

leaves = [v for v in g_union.vertices if g_union.degree(v) == 1 ]

leaves

50


[33, 43, 41, 17, 13, 50, 7, 22, 29]

In [13]:
common_edges = set()
red_edges = set()
blue_edges = set()

stack = [leaves.pop()]
visited = set()

while stack:

    v = stack.pop()
    visited.add(v)

    for w in g_union.adjacent_to(v):
        if red.has_edge(v, w) and blue.has_edge(v, w):
            common_edges.add(UEdge(v, w))

        elif red.has_edge(v, w):
            red_edges.add(UEdge(v, w))

        elif blue.has_edge(v, w):
            blue_edges.add(UEdge(v, w))

        if w not in visited:
            stack.append(w)
        

In [14]:
print(len(common_edges))
print(len(red_edges))
print(len(blue_edges))

29
20
20


In [15]:
for v, u in red_edges:
    if (v, u) in common_edges or (u, v) in common_edges:
        print(v,u)

In [16]:
for v, u in blue_edges:
    if UEdge(v, u) in common_edges :
        print(v,u)

In [17]:
for v, u in red_edges:
    if UEdge(v, u) in blue_edges:
        print(v,u)

### Determinar subestruturas



In [18]:
instance_file = 'steinb5.txt'

stpg = read_problem('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary', instance_file)

In [72]:
generate = GenerateBasedPrimRST(stpg)

In [73]:
red = generate()
blue = generate()

In [74]:
child = UGraph() # g_intersection
red_only = UGraph()
blue_only = UGraph()


for v, u in red.gen_undirect_edges():
    if blue.has_edge(v, u):
        child.add_edge(v,u)
    else:
        red_only.add_edge(v,u)

for v, u in blue.gen_undirect_edges():
    if not red.has_edge(v, u):
        blue_only.add_edge(v, u)

intersect_nodes = set(red_only.vertices) & set(blue_only.vertices)

len(intersect_nodes)

21

In [75]:
class Partition:

    def __init__(self):
        self.edges = set()
        self.portal = set()
        self.cost = 0

    def add(self, v, w):
        self.edges.add(UEdge(v, w))

In [76]:
# red_only.add_edge(41, 40)
# red_only.remove_edge(41, 40)


In [77]:
def display(a, b, back_edge=False):
    if back_edge:
        print(a, b, "<back edge>")
    else:
        print(a, b)

In [78]:
from random import shuffle

In [89]:
red_only.add_edge(35, 30)
red_only.add_edge(36, 42)

In [101]:
'''Esse programa coloca um vértice na stack  repetidas vezes.
Desta forma o algoritmo segue o comportamento esperado pela DFS.
A desvantagem é que  o laço "for w in subgraph.adjacent_to(v):" e a 
verificação que se segue "if w not in visited:" serão repetidas desnecessariamente.

Considere o grafo com as arestas:
 4  -- 28
 4  -- 49
 28 -- 49
 28 -- 39
 39 -- 49
 39 -- 20
 20 -- 49

Por exemplo:
 Se iniciarmos pelo vértice 4, o vértice adjacente 49 será colocado na stack.
 O vértice 49 vai ser colocado outras 3 vezes na stack considerando as visitas 
 dos vértices 28, 39 e 20. 

 E se começarmos pelo vértice 49?
'''
nodes = list(intersect_nodes)

shuffle(nodes)

visited = set()
    
def visit(start, subgraph):

    stack = [start]

    while stack:
        v = stack.pop()
        if v not in visited:
            visited.add(v)
            for w in subgraph.adjacent_to(v):
                if w not in visited:
                    stack.append(w)
                    display(v, w)
                # elif w in stack and w != stack[-1]:
                #     display(v, w, back_edge=True)
        # print(stack)

for s in nodes :
    if s not in visited:
        visit(s, red_only)
        print(10*'-')

20 3
3 34
34 37
37 4
4 24
4 2
2 47
----------
36 32
36 42
42 32
32 1
32 30
30 35
35 1
----------
48 25
----------
12 17
17 21
17 46
46 6
46 38
46 7
21 9
21 49
----------
23 43
23 31
----------


In [92]:
# red_only.add_edge(39, 28)
# red_only.add_edge(39, 49)
# red_only.add_edge(28, 49)

# red_only.remove_edge(39, 28)
# red_only.remove_edge(39, 49)
# red_only.remove_edge(28, 49)

In [93]:
# nodes = list(child.vertices)

intersect_nodes = set(red_only.vertices) & set(blue_only.vertices)
nodes = list(intersect_nodes)

visited = set()
nro_components = 0 

def dfs(subgrah, s):
    # visited  = set()
    stacked = set()
    # nro_components = 0

    def P(subgrah, v, u=None):
        
        visited.add(v)
        stacked.add(v)

        for w in subgrah.adjacent_to(v):
            if w not in visited:
                display(v, w)
                P(subgrah, w, v)
            elif w in stacked and w != u :
                display(v, w, back_edge=True)
        
        stacked.remove(v)
    
    P(subgrah, s)

for s in nodes:
    if s not in visited:
        dfs(red_only, s)
        nro_components += 1
        print(5*'- ')

1 32
32 42
42 36
36 32 <back edge>
32 30
30 35
35 1 <back edge>
- - - - - 
2 4
4 24
4 37
37 34
34 3
3 20
2 47
- - - - - 
6 46
46 17
17 12
17 21
21 9
21 49
46 38
46 7
- - - - - 
23 43
23 31
- - - - - 
25 48
- - - - - 


In [86]:
print(nro_components)

5
